In [544]:
import requests
import json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import os
import webbrowser
import folium
from folium.features import DivIcon
from folium import plugins
print(folium.__version__)

0.12.1.post1


In [560]:
state_geo = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/juso/2015/json/seoul_municipalities_geo_simple.json'

In [561]:
raw_data = pd.read_json(state_geo)
data = [
    ['도봉구', '11320', 127.03278382986406, 37.667199069165044],
    ['은평구', '11380', 126.92388486168254, 37.61600184192064],
    ['동대문구', '11230', 127.05326118159924, 37.58362488095133],
    ['동작구', '11590', 126.95219874667816, 37.50092869817122],
    ['금천구', '11545', 126.89911181670232, 37.46640549118321],
    ['구로구', '11530', 126.86939217118206, 37.49824594332609],
    ['종로구', '11110', 126.98444920906223, 37.59081035058071],
    ['강북구', '11305', 127.0215254728531, 37.632046746187775],
    ['중랑구', '11260', 127.09211952814929, 37.600870218534695],
    ['강남구', '11680', 127.07283998849552, 37.48909545646018],
    ['강서구', '11500', 126.81414741792146, 37.560886482642864],
    ['중구', '11140', 126.99506018214162, 37.558371750266666],
    ['강동구', '11740', 127.14885555264618, 37.551717380107696],
    ['광진구', '11215', 127.08974321215258, 37.55189624394916],
    ['마포구', '11440', 126.91488843257431, 37.55228785521782],
    ['관악구', '11620', 126.93583046752502, 37.47409667026501],
    ['서초구', '11650', 127.0339660401257, 37.46566083673712],
    ['성북구', '11290', 127.02240243094009, 37.60011928159995],
    ['노원구', '11350', 127.0775512372282, 37.64238136613501],
    ['송파구', '11710', 127.12101213187754, 37.50039326183675],
    ['성동구', '11200', 127.03850383263077, 37.55278841153847],
    ['서대문구', '11410', 126.94351606661385, 37.57925155480199],
    ['양천구', '11470', 126.8652013125735, 37.52284379192651],
    ['영등포구', '11560', 126.90733526751012, 37.523154626882516],
    ['용산구', '11170', 126.97963765655432, 37.53836767596737]
]
data

[['도봉구', '11320', 127.03278382986406, 37.667199069165044],
 ['은평구', '11380', 126.92388486168254, 37.61600184192064],
 ['동대문구', '11230', 127.05326118159924, 37.58362488095133],
 ['동작구', '11590', 126.95219874667816, 37.50092869817122],
 ['금천구', '11545', 126.89911181670232, 37.46640549118321],
 ['구로구', '11530', 126.86939217118206, 37.49824594332609],
 ['종로구', '11110', 126.98444920906223, 37.59081035058071],
 ['강북구', '11305', 127.0215254728531, 37.632046746187775],
 ['중랑구', '11260', 127.0921195281493, 37.600870218534695],
 ['강남구', '11680', 127.07283998849552, 37.48909545646018],
 ['강서구', '11500', 126.81414741792146, 37.560886482642864],
 ['중구', '11140', 126.99506018214161, 37.558371750266666],
 ['강동구', '11740', 127.14885555264618, 37.551717380107696],
 ['광진구', '11215', 127.08974321215258, 37.55189624394916],
 ['마포구', '11440', 126.91488843257432, 37.55228785521782],
 ['관악구', '11620', 126.93583046752502, 37.47409667026501],
 ['서초구', '11650', 127.0339660401257, 37.46566083673712],
 ['성북구', '1

In [570]:
df = pd.DataFrame(data, columns=['name', 'code', 'x', 'y'])
df = df.sort_values(by=['name']).reset_index(drop=True)
df.head()

,name,code,x,y
0,강남구,11680,127.072840,37.489095
1,강동구,11740,127.148856,37.551717
2,강북구,11305,127.021525,37.632047
3,강서구,11500,126.814147,37.560886
4,관악구,11620,126.935830,37.474097


In [571]:
def dist(tx, ty, dx, dy):
    return math.sqrt((tx-dx)**2 + (ty-dy)**2)
    
def min_dist(target_x, target_y):
    res = {}
    _min_dist = 1e10
    for i in range(len(df)):
        name, code, x, y = df.iloc[i]
        _dist = dist(target_x, target_y, x, y)
        if _min_dist > _dist: 
            _min_dist = _dist
            res['name'] = name
            res['code'] = code
            res['x'] = x
            res['y'] = y
    return res

In [572]:
df_hotel = pd.read_excel("hotel.xlsx").drop(["Unnamed: 0"], axis=1)
df_corona = pd.read_csv("corona.csv", encoding='euc-kr').drop(["Unnamed: 0"], axis=1)
df_corona = df_corona.drop([4, 26]).reset_index(drop=True)    # '전체' 제거
df_corona.columns = ["name", "value"]

# display(df_hotel.head())
for i in range(len(df_hotel)):
    pos = df_hotel.iloc[i][2]
    if pos is np.nan:
        pos = min_dist(df_hotel.x[i], df_hotel.y[i])['name']
    else:
        pos = pos.split()[1]
    df_hotel.pos[i] = pos
# df_hotel[df_hotel.pos == '강남구']

display(df_hotel)
display(df_corona)
        

/tmp/ipykernel_34075/303496151.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hotel.pos[i] = pos


,x,y,pos
0,126.810277,37.560664,강서구
1,126.810317,37.560657,강서구
2,126.810326,37.560790,강서구
3,126.823284,37.558691,강서구
4,126.815276,37.573113,강서구
...,...,...,...
2194,127.138080,37.538250,강동구
2195,127.142254,37.541107,강동구
2196,127.138220,37.537677,강동구
2197,127.137600,37.538121,강동구


,name,value
0,구로구,6272
1,성동구,3480
2,강동구,5134
3,동대문구,4859
4,노원구,5101
5,광진구,4221
6,송파구,7886
7,서대문구,3322
8,관악구,6691
9,강남구,8176


In [573]:
se_pos_value = df_hotel.pos.value_counts()

state_data = df_corona.sort_values(by=['name']).reset_index(drop=True)

bubble_data = pd.DataFrame([se_pos_value.index, se_pos_value]).drop(25, axis=1).transpose()
bubble_data.columns = ['name', 'value']
bubble_data = bubble_data.sort_values(by=['name']).reset_index(drop=True)
display(bubble_data)

df['value'] = bubble_data.value
display(df)
# display(state_data.value)
# display(df)

,name,value
0,강남구,573
1,강동구,34
2,강북구,28
3,강서구,71
4,관악구,36
5,광진구,44
6,구로구,42
7,금천구,11
8,노원구,22
9,도봉구,8


,name,code,x,y,value
0,강남구,11680,127.072840,37.489095,573
1,강동구,11740,127.148856,37.551717,34
2,강북구,11305,127.021525,37.632047,28
3,강서구,11500,126.814147,37.560886,71
4,관악구,11620,126.935830,37.474097,36
5,광진구,11215,127.089743,37.551896,44
6,구로구,11530,126.869392,37.498246,42
7,금천구,11545,126.899112,37.466405,11
8,노원구,11350,127.077551,37.642381,22
9,도봉구,11320,127.032784,37.667199,8


In [577]:
# Initialize the map:
m = folium.Map(location=[37.562225, 126.978555], tiles="cartodbpositron", zoom_start=12)

m.choropleth(
    geo_data=state_geo,
    name='코로나 확진자 수',
    data=state_data,
    columns=['name', 'value'],
    key_on='feature.properties.SIG_KOR_NM',
    color = 'gray',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name = ''
)

for i in range(len(df)):
    name, code, x, y, count = df.iloc[i]
    value = count
    
    folium.CircleMarker(
        [y, x],
        radius=value/10,
        fill_opacity=0.2,
        opacity=0.5,
        color='blue',
        fill_color='white',
        popup = folium.Popup('<div style="font-size: 10pt; width:50px; text-align:center">%s</div>' % name)
        ).add_to(m)

    folium.map.Marker(
        [y, x],
        icon=DivIcon(
            icon_size=(80,60),
            icon_anchor=(39, 27),
            html='<div style="font-size: 20px;text-align:center;">%s<br>%d</div>' % (name, count),
            )
        ).add_to(m)

m

# folium.LayerControl().add_to(m)

# Save to html
m.save('bubble.html')
# webbrowser.open_new("folium_kr.html")